In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("hw07.ipynb")

<a id="top"></a>

# Homework 7: IMDb

We will use SQL to dive deep into the Internet Movie Database (IMDb) and answer different questions involving movies, actors, and movie ratings.

## Due Date: Monday, November 17th, 11:59 PM PDT

You must submit this assignment to Pensieve by the on-time deadline, Monday, November 17th, 11:59 PM PDT. Please read the syllabus for the Slip Day policy. No late submissions beyond what is outlined in the Slip Day policy will be accepted. **We strongly encourage you to submit your work to Pensieve several hours before the stated deadline.** This way, you will have ample time to reach out to staff for support if you encounter difficulties with submission. While course staff is happy to help guide you with submitting your assignment ahead of the deadline, we will not respond to last-minute requests for assistance.

Please read the instructions carefully when submitting your work to Pensieve. 


##  Collaboration Policy

Data science is a collaborative activity. While you may talk with others about the homework, we ask that you **write your solutions individually**. If you do discuss the assignments with others, please **include their names** below.

**Collaborators**: *list collaborators here*

## Grading 

Grading is broken down into autograded answers and free response. For autograded answers, the results of your code are compared to provided and/or hidden tests. For free response, readers will evaluate how well you answered the question and/or fulfilled the requirements of the question.

<!--
<details>
    <summary>[Click to Expand] <b>Scoring Breakdown</b></summary>-->
|Question| Manual | Points|
|---|---|---|
|1a | No | 2 |
|1b | No | 2 |
|2 | No | 3 |
|3 | No | 3 |
|4 | No | 4 |
|5 | Yes | 2 |
|Total | 1 |16 |
</details>

## Before You Start

For each question in the assignment, please write down your answer in the answer cell(s) right below the question. 

We understand that it is helpful to have extra cells breaking down the process towards reaching your final answer. If you happen to create new cells below your answer to run code, **NEVER** add cells between a question cell and the answer cell below it. It will cause errors when we run the autograder, and it will sometimes cause a failure to generate the PDF file.

**Important note: Please only have ONE active SQL database running at once on DataHub, meaning do not have multiple homework/lab/lecture notebooks running at the same time**

**Important note: The local autograder tests will not be comprehensive. You can pass the automated tests in your notebook but still fail tests in the autograder.** Please be sure to check your results carefully.


### Debugging Guide
If you run into any technical issues, we highly recommend checking out the [Data 100 Debugging Guide](https://ds100.org/debugging-guide/). In this guide, you can find general questions about Jupyter notebooks / Datahub, Pensieve, common SQL errors, and more.

In [ ]:
# Run this cell to set up your notebook; no further action is needed
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import sqlalchemy
from pathlib import Path
import sql

plt.style.use('fivethirtyeight') # Use plt.style.available to see more styles
sns.set()
sns.set_context("talk")
np.set_printoptions(threshold=5) # Avoid printing out big matrices
%matplotlib inline
%load_ext sql

<br/><br/>

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />

## The IMDB (mini) Dataset

We will explore a miniature version of the [IMDb Dataset](https://www.imdb.com/interfaces/). This is the same dataset that we used for this week's lab.

**Caution: Be careful with large SQL queries!!** You may need to reboot your Jupyter Hub instance if it stops responding. To avoid printing out 100k-sized tables, we've adjusted the display limit to ensure that the tables displayed are truncated to 20 rows (though they may contain more rows in reality).

In [ ]:
%config SqlMagic.displaylimit = 20
%config SqlMagic.style = 'DEFAULT'

Let's set up the SQL database.

## Question 1 - Setup

Please just run the cells below to set up your SQL database and the autograder.

In [ ]:
import duckdb

In [ ]:
# Run this cell to connect to the database
conn = duckdb.connect()
conn.query("INSTALL sqlite")

In [ ]:
%sql duckdb:///data/imdbmini.db

Let's take a look at the table schemas:

In [ ]:
%%sql
-- just run this cell --
SELECT * FROM sqlite_master WHERE type='table';

From running the above cell, we see the database has 4 tables: `Name`, `Role`, `Rating`, and `Title`.

<details open>
    <summary>[<b>Click to Expand</b>] See descriptions of each table's schema below. You can also find it in the `schemas.txt` file, which is in the same directory as this notebook. We have only included descriptions for columns that could be of potential use in this homework. </summary>
    
**`Name`** – Contains the following information for names of people.
    
- nconst (BIGINT) - alphanumeric unique identifier of the name/person
- primaryName (VARCHAR) - name by which the person is most often credited
- birthYear (VARCHAR) - in YYYY format
- deathYear (VARCHAR) - in YYYY format
- primaryProfession (VARCHAR) – array of the top-3 professions of the person

    
    
**`Role`** – Contains the principal cast/crew for titles.
    
- tconst (BIGINT) - alphanumeric unique identifier of the title
- ordering (VARCHAR) - a number to uniquely identify rows for a given tconst
- nconst (BIGINT) - alphanumeric unique identifier of the name/person
- category (VARCHAR) - the category of job that person was in
- characters (VARCHAR) - the name of the character played if applicable, else '\\N'
    
**`Rating`** – Contains the IMDb rating and vote information for titles.
    
- tconst (BIGINT) - alphanumeric unique identifier of the title
- averageRating (VARCHAR) – weighted average of all the individual user ratings
- numVotes (VARCHAR) - number of votes (i.e., ratings) the title has received
    
**`Title`** - Contains the following information for titles.
    
- tconst (BIGINT) - alphanumeric unique identifier of the title
- titleType (VARCHAR) -  the type/format of the title
- primaryTitle (VARCHAR) -  the more popular title / the title used by the producers on promotional materials at the point of release
- isAdult (VARCHAR) - 0: non-adult title; 1: adult title
- startYear (VARCHAR) - represents the release year of a title.
- runtimeMinutes (VARCHAR) - primary runtime of the title, in minutes
- genres (VARCHAR) – array that includes up to three genres associated with the title
    
</details>

<br/><br/>
From the above descriptions, we can conclude the following:
* `Name.nconst` and `Title.tconst` are primary keys of the `Name` and `Title` tables, respectively.
* `Role.nconst` and `Role.tconst` are **foreign keys** that point to `Name.nconst` and `Title.tconst`, respectively.

For a refresher on how to write SQL queries in Jupyter, feel free to reference Lab 10. The main thing to keep in mind is that you can directly write your query in place of the ellipsis under `%%sql --save query_q`. **Please do not edit this line.** 

For example, we can set `query_example` to the output of the following SQL query and directly see what it contains: 

In [ ]:
%%sql --save query_example
-- This is a one-line SQL comment.
/* This is a multi-line
   SQL comment. */
SELECT * 
FROM name
LIMIT 15;

<br/>

---

### Question 1a  
Let's determine whether our database includes information going back to the early days of cinema, or just more recent data.

List the **5 oldest movie titles** by `startYear` and then `primaryTitle` both in **ascending** order. The output should contain the `startYear`, `primaryTitle`, and `titleType`. In this homework, we define a movie as having `titleType='movie'`. Keep this in mind for later questions as well.
                                                                                                                               

In [ ]:
%%sql --save query_q1a

...

In [ ]:
# Run this cell for grading purposes. 
# No further action is required. 
query = %sqlcmd snippets query_q1a
res_q1a = pd.read_sql(query, "duckdb:///data/imdbmini.db")

In [ ]:
grader.check("q1a")

<br/>

---

### Question 1b

Next, let's calculate the distribution of movies by year. Write a query that returns the **total** number of movie titles for each `startYear` in the `Title` table as `total`. Order your final results by the `startYear` in **ascending** order. As in `q1a`, remember that movies are defined as having `titleType='movie'`.

The first few records of the table should look like the following (but you should compute the entire table):


|startYear|total|
|------:|-----:|
| 1915|1|
| 1920|1|
| 1921|1|
| 1922|1|
| ...|...|



In [ ]:
%%sql --save query_q1b

...

In [ ]:
# Run this cell for grading purposes. 
# No further action is required. 
query = %sqlcmd snippets query_q1b
res_q1b = pd.read_sql(query, "duckdb:///data/imdbmini.db")

In [ ]:
grader.check("q1b")

<br/>

The following cell should generate an interesting plot of the number of movies that premiered each year. Notice there are fewer movies premiering from the 1920s to the late 1940s. Why might that be? *This question is rhetorical; you do not need to write your answer anywhere.*

In [ ]:
# Run this call to generate the bar plot; no further action is needed
px.bar(res_q1b, x="startYear", y="total", 
        title="Number of movies premiered each year", 
        width=900, height=400)

<br/>

---

## Question 2

Write a SQL query to determine the **movie actors** with the highest total number of movies. The term **movie actor** is defined as anyone with an "actor" or "actress" job category in a "movie" title type released after 1980. Your SQL query should output exactly two fields named `name` (the movie actor’s name) and `total` (the number of movies the movie actor appears in). Order the records by `total` in descending order, and break ties by ordering by `name` in ascending order. **Only include the first 20 rows in your final query.**

Your result should look something like this (but without `????`):

| name | total |
|-----:|-----:|
| ???? | 58 |
| ???? | 54 |
| ???? | 53 |
| ???? | 49 |
| ???? | 46 |
| ???? | 43 |
| ???? | 41 |
| ???? | 40 |
| ???? | 40 |
| ???? | 39 |

**Notes**: 
* **The query should take < 2 minutes to run.**
* Sometimes Python gets confused and colors some SQL queries red; *don't worry if the SQL coloring doesn't match what you'd expect*. As long as it runs, it's fine.

**Hints**:

* Before writing your query, you may wish to review the table descriptions given at the start of the assignment to determine where the information you need is stored
* If you want to include a non-aggregate field in the `SELECT` clause, it must also be included in the `GROUP BY` clause.
* When using multiple conditions in a `WHERE` clause, pay attention to the order of operations.

In [ ]:
%%sql --save query_q2

...

In [ ]:
# Run this cell for grading purposes. 
# No further action is required. 
query = %sqlcmd snippets query_q2
res_q2 = pd.read_sql(query, "duckdb:///data/imdbmini.db")

In [ ]:
grader.check("q2")

<br/>

---

## Question 3: The `CASE` Keyword

The `rating` table has the `numVotes` and the `averageRating` for each title. A movie is considered a **"big hit**" if there are more than 100,000 votes for the movie. Which `movie` titles were **"big hits"**? Construct a query that generates the following result:

| isBigHit | total |
|-----:|-----|
| no | ???? |
| yes | ???? |

Where `????` is replaced with the correct values. The row with `no` should have the count for how many movies **are not** big hits, and the row with `yes` should have the count of how many movies **are** big hits.

**Hints**:

* Check the data types of `numVotes` before performing any arithmetic operations.
* You will need to use some type of `JOIN`.
* You may also consider using a `CASE` statement:
    ```
    CASE 
        WHEN ... THEN ...
        ELSE ... 
    END
    ```
    </br>
    
  `CASE` statements are the SQL equivalent of `Python` `if... elif... else` statements. To read up on `CASE`, take a look at the following links:
    - https://mode.com/sql-tutorial/sql-case/
    - https://www.w3schools.com/sql/sql_ref_case.asp


In [ ]:
%%sql --save query_q3

...

In [ ]:
# Run this cell for grading purposes. 
# No further action is required. 
query = %sqlcmd snippets query_q3
res_q3 = pd.read_sql(query, "duckdb:///data/imdbmini.db")

In [ ]:
grader.check("q3")

<br/>

---

## Question 4

Write a SQL query to determine the movie producers with the highest average ratings across all of their movies. Define a **"movie producer"** as anyone with a `producer` job category role in a `movie` title type. Construct a query that generates a table consisting of the **producer's name** (as `name`) and their **average producer rating** (as `producerRating`), computed by rescaling the ratings for movies they produced by the number of votes received by each movie. After rescaling the score for each movie, divide the summation of all rescaled ratings by the total number of votes received by each producer. The formula is below:

$$
\text{producerRating} = 
\frac{\sum_m (\texttt{averageRating}[m] * \texttt{numVotes}[m] )}{\sum_m \texttt{numVotes}[m]}
$$

To make clear the summation, "m" refers to a particular movie that a producer worked on. Thus, the summation over "m" refers to the summation across all movies that a particular producer contributed to.

In addition to the above, only consider ratings where there are **at least 22,500** votes and only consider movie producers that have **at least 20 rated movies**. Present the producers with the **top 10** `producerRating` in **descending** order and break ties alphabetically using the producer's name.

The results should look something like this but without the `????`, and with higher rating precision.

| name            | producerRating |
|-----------------|----------------|
| ???             | 7.76...        |
| ???             | 7.62...        |
| ???             | 7.59...        |
| ???             | 7.43...        |
| ???             | 7.41...        |
| ???             | 7.35...        |
| ???             | 7.30...        |
| ???             | 7.27...        |
| ???             | 7.25...        |
| ???             | 7.24...        |

**Hint**: Check the data types of `numVotes` and `averageRating` before performing any arithmetic operations.

**Notes**:
* ***The query should take < 3 minutes to run.***
* DO NOT cast `averageRating` **as an integer**. Doing so reduces the precision of the resulting values, so your table may not match up exactly with what is shown below.
* If a producer has multiple `role` listings for a movie, then that movie will have a bigger impact on the overall average (this is desired).


In [ ]:
%%sql --save query_q4

...

In [ ]:
# Run this cell for grading purposes. 
# No further action is required. 
query = %sqlcmd snippets query_q4
res_q4 = pd.read_sql(query, "duckdb:///data/imdbmini.db")

In [ ]:
grader.check("q4")

<br/>

---

# 🤖 Question 5: Open-Ended Question

Welcome to another **open-ended question**.


> If you have any feedback on this open-ended question, or any other homework question in Data 100, we encourage you to share your thoughts using the [content feedback form](https://docs.google.com/forms/d/e/1FAIpQLScGQhiTz1qn5gsyYUu1Be5Yz0Z_kplIRR_os8UGMOAO8cc-yQ/viewform). You can also post to Ed.

Grading on open-ended questions is simple: **Clear evidence of thoughtfulness and effort will always receive full credit**. If your response is especially well-developed or creative, we may ask for permission to share it with the rest of the class so others can be inspired by your work! Underdeveloped ideas will receive half credit. Trivial or missing responses will receive no credit. We expect the vast majority of students to receive full credit.

**SETUP**: You are a data scientist working for Streaming Co., a new movie streaming company. Your manager wants to understand the changes in movie preferences over time to help them come up with the next blockbuster movie.

**TASK**: Your task is to create 1 visualization to help your manager understand the changes in movie trends over time and suggest at least one thing you might want to include in this blockbuster movie.

> You might be wondering: What counts as a change in movie trends? This is completely up to you. This is a chance for you to flex your creativity. There is no single correct answer. Just be sure to clearly explain what your visualization shows and how it supports your idea for the next blockbuster movie. Remember that a major part of data science is defining what success actually means.

> **Note**: This dataset was last updated in 2021 so if you want to do some exploration of trends over time, you can imagine we are currently in 2022 and this is our most up-to-date dataset.
    
> **Additional Instructions**: Most of you code should be SQL, the only non-SQL code should just be turning your `DataFrame` into a visualization. All joins, grouping, aggregation, filtering rows, selecting columns, ordering rows, and limiting rows should be done in SQL.


**Your answer should consist of the following**:
1. A **single visualization**. The contents of this visualization are ultimately up to you. 
2. A write up of **4–10 sentences** stating your idea for the next blockbuster movie, an explanation of why you think doing this will help you make the next blockbuster movie, and **one possible counterargument** as to why your idea might not lead to a blockbuster movie. Your explanation must reference the visualization.

**Note**: For this question we are **not** accepting `DataFrame`s as visualizations.
                                                                                                                                                          
For this task, you may use: 
- All the tables in the IMDb database: `Name`, `Role`, `Rating`, and `Title`.
- Any SQL, `pandas`, `regex`, `matplotlib`, `seaborn` covered in class.
- (Optional) External resources (e.g., AI/LLMs, websites, datasets, or other Python libraries/packages).

                                                      
*Important exception to existing course policies: **FOR THIS QUESTION ONLY**, you are allowed to [vibe code](https://en.wikipedia.org/wiki/Vibe_coding). In other words, the code you use to generate responses can be generated by a large-language model (LLM), like Gemini or ChatGPT. However, the most important component of this question is not the code—it's the presentation and persuasiveness of your results. **If you copy-and-paste default output from an LLM on this question, there is a good chance that your submission will look identical or near-identical to many other students**. While we expect many answers to this question to have similarities, obvious default output will receive no credit. Spend time thinking about the presentation of your results.* 

> **Disclaimer**: As Data Science students, you should be aware of important limitations and broader considerations when it comes to the use of LLMs. 
> - LLMs do not guarantee factual accuracy and they are known to hallucinate (generate fabricated or misleading information). 
> - LLMs are trained on large datasets that can reflect and reproduce biases in race, gender, culture, and ideology. 
> - The use of LLMs may involve the sharing of sensitive and personal information.

    

> **IMPORTANT**: If you have any questions, please read through the [**FAQS**](https://docs.google.com/document/d/1JDNQ9Lnynft-WA3WG3CYfP2yLGNqARJg9PcOUkbEBO8/edit?usp=sharing) first. If you can't find the answer to your question there, feel free to ask your question on Ed.


### SUBMISSION INSTRUCTIONS

**Please read these carefully**
1. In the **SQL Scratch cell** write your SQL query to create the `DataFrame` you will use in your visualization.
2. Run the next to save the result of your SQL query into a `pandas` `DataFrame` called `res_q5`
3. In the next scratch cell called **Visualization Scratch cell**, write any code you need to create your **single visualization**.
4. Once you are happy with your visualization, **remove** any time you wrote `plt.show()` as it prevents the visualization from being saved as an image.
5. Run the scratch cell one last time without `plt.show()`.
> `plt.savefig("final_viz.png", dpi=300, bbox_inches="tight")` at the bottom of the scratch cell saves your visualization to `final_viz.png`.
6. **Comment out all code** in the visualization scratch cells. **Including** `plt.savefig("final_viz.png", dpi=300, bbox_inches="tight")`.
7. Run the next cell to display your visualization. Make sure this is the correct visualization you want as part of your final deliverable.
8. Set `commented_out` to `True`.
9. In the markdown cell below, complete your write-up in 4–10 sentences.

**Note**: Please check your visualization and write-up appears under Q5 in the *Homework 7 Written* Pensieve assignment.

<!-- BEGIN QUESTION -->



In [ ]:
%%sql --save query_q5
-- SQL Scratch Cell
...

In [ ]:
# DO NOT EDIT, JUST RUN THIS CELL

query = %sqlcmd snippets query_q5
res_q5 = pd.read_sql(query, "duckdb:///data/imdbmini.db")

In [ ]:
# Visualization Scratch Cell
# Feel free to do your rough work here
# (Optional) Feel free to import any extra libraries or packages here
# Remember to comment out your code for step 5

...

# Remember to comment out this code for step 5
# Until then, do not edit this code
plt.savefig("final_viz.png", dpi=300, bbox_inches="tight")

In [ ]:
# DO NOT EDIT THIS CELL
# Run this cell and make sure the image that appears is the visualization you want

from IPython.display import Image

Image("final_viz.png")

In [ ]:
# Set commented_out = True once you have commented out all your code from the scratch cells

commented_out = ...

_Type your answer here, replacing this text._

In [ ]:
grader.check("q5")

<!-- END QUESTION -->


<br><br>


<hr style="border: 5px solid #003262;" />

## Helios congratulate you for finishing Homework 7!

<img src="images/helios.jpg" width="300"/>

### Course Content Feedback

If you have any feedback about this assignment or about any of our other weekly, weekly assignments, lectures, or discussions, please fill out the [Course Content Feedback Form](https://docs.google.com/forms/d/e/1FAIpQLScGQhiTz1qn5gsyYUu1Be5Yz0Z_kplIRR_os8UGMOAO8cc-yQ/viewform?usp=dialog). Your input is valuable in helping us improve the quality and relevance of our content to better meet your needs and expectations!

### Submission Instructions

Below, you will see a cell. Running this cell will automatically generate a zip file with your autograded answers. Once you submit this file to the Homework 7 Coding assignment on Pensieve, Pensieve will automatically submit a PDF file with your written answers to the Homework 7 Written assignment. If you run into any issues when running this cell, feel free to check the ["Why did a Pensieve test fail when all the Jupyter notebook’s tests passed?"](https://ds100.org/debugging-guide/autograder_gradescope/autograder_gradescope.html#why-did-a-gradescope-test-fail-when-all-the-jupyter-notebooks-tests-passed) section of the Data 100 Debugging Guide.

If there are issues with automatically generating the PDF, try downloading the notebook as a PDF by clicking on `File -> Save and Export Notebook As... -> PDF`. If that doesn't work, download the notebook as an HTML file by clicking on `File -> Save and Export Notebook As... -> HTML`, open the HTML file in your browser, and Ctrl + P or Cmd + P (Mac) to save the file as a PDF. Alternatively, you can  manually take screenshots of your answers to the manually graded questions and submit a PDF containing those. 

**You are responsible for ensuring your submission follows our requirements and that everything was generated and submitted correctly. We will not be granting regrade requests nor extensions to submissions that don't follow instructions.** If you encounter any difficulties with submission, please don't hesitate to reach out to staff prior to the deadline. 

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export(run_tests=True)